In [1]:
import pandas as pd
from fastavro import writer, parse_schema, reader
pd.set_option('display.max_columns', None)

In [2]:
basepath = '../../datasets/preprocessed_datasets/retailrocket/'
customer_data = pd.read_pickle(basepath +'visitor_actions.pkl')
customer_data

,visitorid,products_viewed,num_views,num_add2cart,num_transaction
0,693516,1,1,2,0
1,829044,1,1,0,0
2,652699,1,1,0,0
3,1125936,1,1,0,0
4,1149227,1,1,0,0
...,...,...,...,...,...
1407575,998517,1,1,0,0
1407576,472345,1,1,0,0
1407577,1207677,1,1,0,0
1407578,255126,1,1,0,0


In [3]:
schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "IS_VISITOR",
            "type": "boolean"
        }
    ],
    "version": "1.0"
}
parsed_schema = parse_schema(schema)

In [4]:
customer_data = customer_data.drop(['products_viewed', 'num_views', 'num_add2cart', 'num_transaction'], axis=1).rename({'visitorid': 'USER_ID'}, axis=1)
customer_data['IS_VISITOR'] = 'true'
customer_data['USER_ID'] = customer_data['USER_ID'].astype(int).astype(str)
display(customer_data)

,USER_ID,IS_VISITOR
0,693516,true
1,829044,true
2,652699,true
3,1125936,true
4,1149227,true
...,...,...
1407575,998517,true
1407576,472345,true
1407577,1207677,true
1407578,255126,true


In [5]:
customer_data.to_csv(basepath + '/aws/customer_data.csv', index=False, header=True)

In [4]:
item_data = pd.read_pickle(basepath +'item_data_extracted.pkl')
item_data

,itemid,categoryid,available,properties,property_values
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189,...","[1276750, n3.168 1144008, n15360.000 628176 n1..."
1,1,1114,1,"[0, 1036, 112, 159, 185, 202, 227, 238, 280, 2...","[769062, 1154859, 679677, 519769, 769062, 1726..."
2,2,1305,0,"[1063, 112, 159, 202, 282, 283, 318, 332, 364,...","[n396.000 145688, 679677, 519769, 648485 n6000..."
3,3,1171,0,"[1025, 1080, 112, 159, 202, 227, 250, 283, 30,...","[769062, 769062, 679677, 519769, 261419, 13057..."
4,4,1038,0,"[112, 115, 159, 202, 227, 28, 283, 33, 364, 59...","[679677, n24.000, 519769, 371058 71429, 588652..."
...,...,...,...,...,...
417048,466862,1376,0,"[112, 139, 159, 186, 202, 227, 28, 283, 29, 34...","[679677, 769062, 519769, 575816 n432.000, 7173..."
417049,466863,173,0,"[1014, 112, 159, 202, 227, 283, 364, 400, 47, ...","[1075463, 679677, 519769, 1194687 550565 24255..."
417050,466864,373,1,"[1036, 112, 152, 159, 202, 227, 230, 283, 348,...","[1154859, 679677, 1071492, 519769, 1262739 205..."
417051,466865,421,0,"[1031, 112, 159, 202, 227, 277, 28, 283, 332, ...","[1088309, 679677, 519769, 150169 780351 820477..."


In [3]:
def list_to_pipe_string(lst):
    return '|'.join(str(elem) for elem in lst)

def convert_bool_to_string(element):
    if element == 1 or element == '1':
        return 'true'
    else:
        return 'false'


def convert_event_type(element):
    if element == 'view':
        return 'watch'
    elif element == 'addtocart':
        return 'add_to_cart'
    elif element == 'transaction':
        return 'purchase' # needs to be called purchased for some recommenders like, bought togheter or bestsellers
    else:
        return 'VIEW'

In [ ]:
display(item_data)

In [5]:
item_data = item_data.rename({'itemid': 'ITEM_ID', 'properties': 'PROPERTY', 'property_values': 'PROPERTY_VALUE', 'categoryid': "CATEGORY_L1", 'available': 'AVAILABLE'}, axis=1)
item_data['PRICE'] = 1.0
#property values cannot be used, because they are too long (most of the time they are the same, but sometimes they are not, but for testing it does not matter much)
#item_data['PROPERTY_VALUE'] = item_data['PROPERTY_VALUE'].apply(list_to_pipe_string)
item_data['ITEM_ID'] = item_data['ITEM_ID'].astype(str)
item_data['CATEGORY_L1'] = item_data['CATEGORY_L1'].astype(str)
item_data['AVAILABLE'] = item_data['AVAILABLE'].apply(convert_bool_to_string)
item_data.drop(['PROPERTY_VALUE'], axis=1, inplace=True)
item_data['PROPERTY'] = item_data['PROPERTY'].apply(list_to_pipe_string)
item_data

NameError: name 'convert_bool_to_string' is not defined

In [10]:
item_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "PROPERTY",
            "type": "string",
            "categorical": 'true'
        },
        {
            "name": "AVAILABLE",
            "type": "boolean"
        },
        {
            "name": "PRICE",
            "type": "float"
        },
        {
            "name": "CATEGORY_L1",
            "type": "string",
            "categorical": 'true'

        }
    ],
    "version": "1.0"
}

In [11]:
item_data.to_csv(basepath + '/aws/item_data.csv', index=False, header=True)

In [4]:
interaction_data = pd.read_pickle(basepath +'events.pkl')
interaction_data

,timestamp,visitorid,event,itemid,transactionid,datetime
0,1433221332117,257597,view,355908,NaN,2015-06-02 07:02:12
1,1433224214164,992329,view,248676,NaN,2015-06-02 07:50:14
2,1433221999827,111016,view,318965,NaN,2015-06-02 07:13:19
3,1433221955914,483717,view,253185,NaN,2015-06-02 07:12:35
4,1433221337106,951259,view,367447,NaN,2015-06-02 07:02:17
...,...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN,2015-08-01 05:13:05
2756097,1438399813142,762376,view,115946,NaN,2015-08-01 05:30:13
2756098,1438397820527,1251746,view,78144,NaN,2015-08-01 04:57:00
2756099,1438398530703,1184451,view,283392,NaN,2015-08-01 05:08:50


In [5]:
interaction_data = interaction_data.drop(['datetime'], axis=1).rename({'visitorid': 'USER_ID', 'itemid': 'ITEM_ID', 'timestamp' : 'TIMESTAMP', 'event': 'EVENT_TYPE', 'transactionid': 'TRANSACTION_ID'}, axis=1)
interaction_data['TRANSACTION_ID'] = interaction_data['TRANSACTION_ID'].fillna(0).astype(int)
interaction_data['EVENT_TYPE'] = interaction_data['EVENT_TYPE'].apply(convert_event_type)

interaction_data

,TIMESTAMP,USER_ID,EVENT_TYPE,ITEM_ID,TRANSACTION_ID
0,1433221332117,257597,watch,355908,0
1,1433224214164,992329,watch,248676,0
2,1433221999827,111016,watch,318965,0
3,1433221955914,483717,watch,253185,0
4,1433221337106,951259,watch,367447,0
...,...,...,...,...,...
2756096,1438398785939,591435,watch,261427,0
2756097,1438399813142,762376,watch,115946,0
2756098,1438397820527,1251746,watch,78144,0
2756099,1438398530703,1184451,watch,283392,0


In [ ]:
display(interaction_data)
{

    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "TRANSACTION_ID",
            "type": "long"
        }
    ],
    "version": "1.0"
}

In [7]:
display(interaction_data[interaction_data['EVENT_TYPE'] == 'purchase'].head())

,TIMESTAMP,USER_ID,EVENT_TYPE,ITEM_ID,TRANSACTION_ID
130,1433222276276,599528,purchase,356475,4000
304,1433193500981,121688,purchase,15335,11117
418,1433193915008,552148,purchase,81345,5444
814,1433176736375,102019,purchase,150318,13556
843,1433174518180,189384,purchase,310791,7244


In [ ]:
display(item_data)

In [15]:
interaction_data.to_csv(basepath + '/aws/transaction_data.csv', index=False, header=True)

results for best sellers, with user id 1233000
461686
213834
119736
312728
7943
445351
48030
248455
420960
17478
320130
409804
416017
46156
369447
268883
257040
334401
441852
37029
318333
9877
546
450082
190000

## Evaluate Results

In [12]:
item_data = pd.read_pickle(basepath +'item_data_extracted.pkl')
interaction_data = pd.read_pickle(basepath +'events.pkl')
display(item_data, interaction_data)

,itemid,categoryid,available,properties,property_values
0,0,209,0,"[1036, 1056, 11, 112, 127, 139, 159, 177, 189,...","[1276750, n3.168 1144008, n15360.000 628176 n1..."
1,1,1114,1,"[0, 1036, 112, 159, 185, 202, 227, 238, 280, 2...","[769062, 1154859, 679677, 519769, 769062, 1726..."
2,2,1305,0,"[1063, 112, 159, 202, 282, 283, 318, 332, 364,...","[n396.000 145688, 679677, 519769, 648485 n6000..."
3,3,1171,0,"[1025, 1080, 112, 159, 202, 227, 250, 283, 30,...","[769062, 769062, 679677, 519769, 261419, 13057..."
4,4,1038,0,"[112, 115, 159, 202, 227, 28, 283, 33, 364, 59...","[679677, n24.000, 519769, 371058 71429, 588652..."
...,...,...,...,...,...
417048,466862,1376,0,"[112, 139, 159, 186, 202, 227, 28, 283, 29, 34...","[679677, 769062, 519769, 575816 n432.000, 7173..."
417049,466863,173,0,"[1014, 112, 159, 202, 227, 283, 364, 400, 47, ...","[1075463, 679677, 519769, 1194687 550565 24255..."
417050,466864,373,1,"[1036, 112, 152, 159, 202, 227, 230, 283, 348,...","[1154859, 679677, 1071492, 519769, 1262739 205..."
417051,466865,421,0,"[1031, 112, 159, 202, 227, 277, 28, 283, 332, ...","[1088309, 679677, 519769, 150169 780351 820477..."


,timestamp,visitorid,event,itemid,transactionid,datetime
0,1433221332117,257597,view,355908,NaN,2015-06-02 07:02:12
1,1433224214164,992329,view,248676,NaN,2015-06-02 07:50:14
2,1433221999827,111016,view,318965,NaN,2015-06-02 07:13:19
3,1433221955914,483717,view,253185,NaN,2015-06-02 07:12:35
4,1433221337106,951259,view,367447,NaN,2015-06-02 07:02:17
...,...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN,2015-08-01 05:13:05
2756097,1438399813142,762376,view,115946,NaN,2015-08-01 05:30:13
2756098,1438397820527,1251746,view,78144,NaN,2015-08-01 04:57:00
2756099,1438398530703,1184451,view,283392,NaN,2015-08-01 05:08:50


In [4]:
best_sellers_for_1233000 = [
    461686, 213834, 119736, 312728, 7943, 445351, 48030, 248455, 420960, 17478,
    320130, 409804, 416017, 46156, 369447, 268883, 257040, 334401, 441852,
    37029, 318333, 9877, 546, 450082, 190000
]


In [10]:
bought_together_item_309858 =  [
    461686, 213834, 119736, 312728, 7943, 445351, 48030, 248455, 420960, 17478,
    320130, 409804, 416017, 46156, 369447, 268883, 334401, 257040, 441852,
    37029, 318333, 9877, 546, 450082, 190000
]

In [13]:
display('user 1233000 bought following itmes', interaction_data[interaction_data['visitorid'] == 1233000])


item_data[item_data['itemid'].isin(best_sellers_for_1233000)]

'user 1233000 bought following itmes'

,timestamp,visitorid,event,itemid,transactionid,datetime
1462976,1430622049851,1233000,view,396707,NaN,2015-05-03 05:00:49


,itemid,categoryid,available,properties,property_values
496,546,1349,1,"[112, 159, 202, 214, 227, 283, 364, 6, 678, 69...","[679677, 519769, 878840 1169762 283006 767591 ..."
7107,7943,398,1,"[1036, 112, 159, 202, 227, 28, 283, 364, 591, ...","[1154859, 679677, 519769, 538364, 588652 10914..."
8820,9877,858,1,"[112, 159, 19, 202, 227, 28, 283, 325, 364, 52...","[679677, 519769, 1297729 n60.000 350726 30603 ..."
15604,17478,1373,1,"[112, 159, 19, 202, 227, 28, 283, 364, 521, 55...","[679677, 519769, 1297729 n36.000 350726 30603 ..."
33069,37029,1483,1,"[1, 1032, 1036, 1037, 1079, 1090, 112, 120, 15...","[1021755, [703585 499075 1277894 1059922 98606..."
41186,46156,720,1,"[112, 159, 202, 28, 283, 348, 364, 6, 678, 680...","[679677, 519769, 309399, 266750, 998633 116584..."
42861,48030,1219,1,"[112, 159, 202, 227, 283, 329, 364, 377, 415, ...","[679677, 519769, 656309, 1324373, 1324373 8960..."
107040,119736,57,1,"[1081, 112, 159, 202, 227, 283, 364, 470, 6, 6...","[769062, 679677, 519769, 1278980 n38496.000, 1..."
169813,190000,5,1,"[1036, 1066, 112, 159, 202, 227, 230, 283, 307...","[616467, 1178208 n1200.000 424566, 679677, 519..."
191082,213834,1163,1,"[1036, 112, 113, 159, 202, 227, 279, 283, 364,...","[1154859, 679677, 426626 703408 1182787, 51976..."


In [11]:
display(item_data[item_data['itemid'] == 309858])
item_data[item_data['itemid'].isin(bought_together_item_309858)]

,itemid,categoryid,available,properties,property_values
276916,309858,799,0,"[1066, 112, 159, 202, 227, 283, 364, 384, 400,...","[n1260.000 424566, 679677, 519769, 125782 n144..."


,itemid,categoryid,available,properties,property_values
496,546,1349,1,"[112, 159, 202, 214, 227, 283, 364, 6, 678, 69...","[679677, 519769, 878840 1169762 283006 767591 ..."
7107,7943,398,1,"[1036, 112, 159, 202, 227, 28, 283, 364, 591, ...","[1154859, 679677, 519769, 538364, 588652 10914..."
8820,9877,858,1,"[112, 159, 19, 202, 227, 28, 283, 325, 364, 52...","[679677, 519769, 1297729 n60.000 350726 30603 ..."
15604,17478,1373,1,"[112, 159, 19, 202, 227, 28, 283, 364, 521, 55...","[679677, 519769, 1297729 n36.000 350726 30603 ..."
33069,37029,1483,1,"[1, 1032, 1036, 1037, 1079, 1090, 112, 120, 15...","[1021755, [703585 499075 1277894 1059922 98606..."
41186,46156,720,1,"[112, 159, 202, 28, 283, 348, 364, 6, 678, 680...","[679677, 519769, 309399, 266750, 998633 116584..."
42861,48030,1219,1,"[112, 159, 202, 227, 283, 329, 364, 377, 415, ...","[679677, 519769, 656309, 1324373, 1324373 8960..."
107040,119736,57,1,"[1081, 112, 159, 202, 227, 283, 364, 470, 6, 6...","[769062, 679677, 519769, 1278980 n38496.000, 1..."
169813,190000,5,1,"[1036, 1066, 112, 159, 202, 227, 230, 283, 307...","[616467, 1178208 n1200.000 424566, 679677, 519..."
191082,213834,1163,1,"[1036, 112, 113, 159, 202, 227, 279, 283, 364,...","[1154859, 679677, 426626 703408 1182787, 51976..."
